In [21]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [22]:
site = 'https://www.hanbit.co.kr'
url = f'{site}/store/books/new_book_list.html'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [23]:
book_list = soup.select('.sub_book_list')
len(book_list)

20

In [24]:
book = book_list[0]
name = book.select_one('.book_tit a').get_text().strip()
name

'하루 한 장 초등과학 365'

In [25]:
author = book.select_one('.book_writer').get_text().strip()
author

'치바 가즈요시'

In [26]:
href = book.select_one('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [27]:
sub_url = site + href
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6694310162'

In [28]:
sub_res = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [29]:
def convert_unit(s):
    return s.replace('<span>','').replace('</span>','').replace('-', '').replace(',','').replace('<span class="pbr">','')

In [30]:
trans = sub_soup.select('ul.info_list span')[1]
trans = convert_unit(str(trans))
trans

'허영은  유우종 (감수) '

In [31]:
date = sub_soup.select('ul.info_list span')[2]
date = convert_unit(str(date))
date = date[2:]
date

'211214'

In [32]:
import re
page = str(sub_soup.select('ul.info_list span')[3])
page = re.sub('[^0-9]','',page) 
page

'416'

In [33]:
fixed_price = sub_soup.select_one('del').get_text()
fixed_price = int(fixed_price[:-1].replace(',',''))
fixed_price

19800

In [34]:
price = sub_soup.select_one('span.pbr strong').get_text().strip()
price = int(convert_unit(price))
price

17820

In [35]:
mile = sub_soup.select('span.pbr')[2]
mile = int(convert_unit(str(mile)).split()[0][:-1])
mile 

990

In [36]:
lines = []

for book in book_list:
    name = book.select_one('.book_tit a').get_text().strip()
    author = book.select_one('.book_writer').get_text().strip()
    
    href = book.select_one('a')['href']
    sub_url = site + href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

    if len(sub_soup.select('ul.info_list span')) == 5:
        trans = str('X')
        date = sub_soup.select('ul.info_list span')[1]
        date = convert_unit(str(date))
        date = date[2:]
        page = str(sub_soup.select('ul.info_list span')[2])
        page = re.sub('[^0-9]','',page)

    elif sub_soup.select('ul.info_list strong')[5].get_text() == '구판정보 :':
        trans = str('X')
        date = sub_soup.select('ul.info_list span')[1]
        date = convert_unit(str(date))
        date = date[2:]
        page = str(sub_soup.select('ul.info_list span')[2])
        page = re.sub('[^0-9]','',page)

    else:
        trans = sub_soup.select('ul.info_list span')[1]
        trans = convert_unit(str(trans))
        date = sub_soup.select('ul.info_list span')[2]
        date = convert_unit(str(date))
        date = date[2:]
        page = str(sub_soup.select('ul.info_list span')[3])
        page = re.sub('[^0-9]','',page) 


    fixed_price = sub_soup.select_one('del').get_text()
    fixed_price = int(fixed_price[:-1].replace(',',''))
    price = sub_soup.select_one('span.pbr strong').get_text().strip()
    price = int(convert_unit(price))
    mile = sub_soup.select('span.pbr')[2]
    mile = int(convert_unit(str(mile)).split()[0][:-1])
    
    lines.append([name, author, trans, date, page, fixed_price, price, mile])

In [37]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판매가', '마일리지'])
df

,제목,저자,역자,출간일,페이지,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,허영은 유우종 (감수),211214,416,19800,17820,990
1,"다시, 배우다",폴 김,X,211210,304,16000,14400,800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,김민성 곽서희,211210,376,22000,19800,1100
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",X,211201,384,18000,16200,900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale",나양 이승배 임윤철,211130,600,36000,36000,1080
5,엑셀 매크로&VBA 업무 공략집,최준선,X,211130,328,22000,19800,1100
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,211126,372,24000,21600,1200
7,돈 되는 기획,김도균,X,211126,240,14500,13050,725
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,X,211125,300,18000,16200,900
9,초등 어휘력이 공부력이다,박명선,X,211125,412,18000,16200,900


In [38]:
from tqdm.notebook import tqdm      #진행률 보여줌

lines = []
for pag in tqdm(range(1,11)):
    url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(pag)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    book_list = soup.select('.sub_book_list')

    for book in book_list:
        name = book.select_one('.book_tit a').get_text().strip()
        author = book.select_one('.book_writer').get_text().strip()
        
        href = book.select_one('a')['href']
        sub_url = site + href
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

        if len(sub_soup.select('ul.info_list span')) == 5:
            trans = str('X')
            date = sub_soup.select('ul.info_list span')[1]
            date = convert_unit(str(date))
            date = date[2:]
            page = str(sub_soup.select('ul.info_list span')[2])
            page = re.sub('[^0-9]','',page)

        elif sub_soup.select('ul.info_list strong')[5].get_text() == '구판정보 :':
            trans = str('X')
            date = sub_soup.select('ul.info_list span')[1]
            date = convert_unit(str(date))
            date = date[2:]
            page = str(sub_soup.select('ul.info_list span')[2])
            page = re.sub('[^0-9]','',page)

        else:
            trans = sub_soup.select('ul.info_list span')[1]
            trans = convert_unit(str(trans))
            date = sub_soup.select('ul.info_list span')[2]
            date = convert_unit(str(date))
            date = date[2:]
            page = str(sub_soup.select('ul.info_list span')[3])
            page = re.sub('[^0-9]','',page) 

        fixed_price = sub_soup.select_one('del').get_text()
        fixed_price = int(fixed_price[:-1].replace(',',''))
        price = sub_soup.select_one('span.pbr strong').get_text().strip()
        price = int(convert_unit(price))
        mile = sub_soup.select('span.pbr')[2]
        mile = int(convert_unit(str(mile)).split()[0][:-1])
        lines.append([name, author, trans, date, page, fixed_price, price, mile])

  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: '대부분 LSTM 모델을 대표적으로 설명하기에 이 대목에서 국뽕이 상승하고 뿌듯했다..'

In [ ]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판매가', '마일리지'])
df

# 9개밖에 안나와요...

,제목,저자,역자,출간일,페이지,정가,판매가,마일리지
0,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,X,211102,96,8000,7200,400
1,제대로 배우는 수학적 최적화,우메타니 슌지,김모세,210930,424,38000,34200,1900
2,스파크를 이용한 자연어 처리,앨릭스 토머스,이창현,210820,440,34000,30600,1700
3,재미있고 빠른 숫자 쓰기 1권 : 0부터 50까지,한빛학습연구회,X,210726,88,7500,6750,375
4,"IT CookBook, 쉽게 배우는 소프트웨어 공학(2판)",김치수,X,210630,560,30000,30000,900
5,"STEM CookBook, 기초 선형대수학(2판)","이광연 , 설한국 , 김진수",X,210607,440,26000,26000,780
6,고성능 파이썬(2판),"미샤 고렐릭 , 이안 오스발트",오현석,210510,528,36000,32400,1800
7,실전 시계열 분석,에일린 닐슨,박찬성,210409,568,38000,34200,1900
8,디노 첫걸음,페르난도 돌리오,우정은,210310,160,17000,15300,850
9,웹 애플리케이션 보안,앤드루 호프먼,최용,210219,372,30000,27000,1500
